In [1]:
!ls

DS_Project_Presentation.pdf README.md
Data                        TEMPLATE_README.md
Movie_Project_Draft.ipynb   Visuals
Notebooks


In [2]:
!ls -a zippedData

ls: zippedData: No such file or directory


# Data Merge & Cleaning

## Import CSV Files

In [3]:
import pandas as pd

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/Users/stephen/opt/anaconda3/envs/learn-env/bin/python"
  * The NumPy version is: "1.19.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/stephen/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 2): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /Users/stephen/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages

In [ ]:
# Create Dataframes
n_b = pd.read_csv('name.basics.csv')

In [ ]:
t_a = pd.read_csv('title.akas.csv')

In [ ]:
t_b = pd.read_csv('title.basics.csv')

In [ ]:
t_p = pd.read_csv('title.principals.csv')

In [ ]:
m_b = pd.read_csv('tn.movie_budgets.csv')

In [ ]:
# Find columns to merge on
print('< n_b >')
print(n_b.info())
print()
print('< t_a >')
print(t_a.info())
print()
print('< t_b >')
print(t_b.info())
print()
print('< t_p >')
print(t_p.info())
print()
print('< m_b >')
print(m_b.info())
print()

Common column names for imdb files are tconst and nconst.

However, we need title and year to merge tn.movie_budgets.csv with imdb dataframes 

## Dataframe Cleaning

Clean the dataframes before merge

### name.basics

In [ ]:
# n_b
# drop irrelevant columns
n_b = n_b.drop('birth_year', 1)
# drop any people who are dead
n_b = n_b[n_b['death_year'].isna()]
# drop death year because no longer relevant
n_b = n_b.drop('death_year', 1)
# drop na values in title id
n_b = n_b[n_b['known_for_titles'].notna()]
# drop na values in primary profession
n_b = n_b[n_b['primary_profession'].notna()]

n_b.info()

### title.akas

In [ ]:
# drop irrelevant columns
t_a = t_a.drop(columns = ['language', 'types', 'attributes'])
# only want original titles to get rid of repeats
is_original_title = t_a['is_original_title'] == 1
is_original_title = t_a[is_original_title]
t_a = is_original_title
# drop irrelevant columns 
t_a = t_a.drop('region', 1)
t_a = t_a.drop('ordering', 1)
# convert column title_id to tconst for merge
t_a['tconst'] = t_a['title_id']
t_a = t_a.drop('title_id', 1)

t_a.info()

### title.basics

In [ ]:
# drop irrelevant column
t_b = t_b.drop('runtime_minutes', 1)
# drop na values in genres 
t_b = t_b[t_b['genres'].notna()]
# drop og title becaus it is not needed
t_b = t_b.drop('original_title', 1)

t_b

In [ ]:
t_b['genres'] = t_b['genres'].str.split(",")
t_b['genres'] = t_b['genres'].tolist()
t_b = t_b.explode('genres')
t_b

In [ ]:
t_b.info()

### title.principals

In [ ]:
# drop irrelevant columns
t_p = t_p.drop(columns = ['characters', 'ordering', 'job'])
# Renamed column category to job
t_p.rename({'category':'job'}, axis = 1, inplace = True)
t_p.info()

In [ ]:
t_p.head()

### tn.movie_budgets

In [ ]:
# Extract month and year from the column release date
# Create column month and year
# Name year column as start_year for merge
pd.to_datetime(m_b['release_date'])
m_b['month'] = pd.DatetimeIndex(m_b['release_date']).month
m_b['start_year'] = pd.DatetimeIndex(m_b['release_date']).year
# drop irrelevant columns
m_b = m_b.drop(columns=['release_date', 'id', 'domestic_gross'], axis = 1)
# rename column movie to title for merge
m_b.rename({'movie' : 'title'}, axis = 1, inplace=True)
m_b.head()

In [ ]:
# Current type of the budget and gross is object
# Remove $ and , and change type to float
m_b[m_b.columns[1:]] = m_b[m_b.columns[1:]].replace('[\$,]', '', regex=True).astype(float)
# Create column profit
m_b['profit'] = m_b['worldwide_gross'] - m_b['production_budget']
# Create return on investment ratio (ROI)
m_b['ROI'] = m_b['profit']*100/m_b['production_budget']
m_b.head()

In [ ]:
m_b.info()

## Data Merge

In [ ]:
# inner merge name.basics with title.principals on column nconst
df1 = n_b.merge(t_p, on = 'nconst', how = 'inner')
df1.head()

In [ ]:
# inner merge previous dataframe with title.akas on column tconst
df2 = df1.merge(t_a, on='tconst', how = 'inner')
df2.head()

In [ ]:
# inner merge previous dataframe with title.basics on column tconst
df3 = df2.merge(t_b, on='tconst', how ='inner')
df3

In [ ]:
# inner merge previous dataframe with tn.movie_budgets on multiple columns (title , start_year) 
final_df = df3.merge(m_b, on=['title', 'start_year'], how = 'inner')
final_df.head()

## Clean Final Dataframe

In [ ]:
# drop irrelevant columns
final_df = final_df.drop(columns = ['is_original_title', 'tconst', 'primary_title', 'nconst','primary_profession','known_for_titles'])
final_df.head()

In [ ]:
# Reorganize the columns
# Make a list of column names
cols = final_df.columns.tolist()
cols

In [ ]:
# Reorganize the columns and apply
cols = ['title','genres','start_year', 'month','primary_name','job','production_budget','worldwide_gross','profit','ROI']
final_df = final_df[cols]
final_df.head()

In [ ]:
# Changing job values
# Check the values inf column job
final_df['job'].value_counts()

In [ ]:
# Actress& Self is basically same job with actor  
# Therefore, witch those values to actor in case we use the column
final_df.iloc[final_df['job'] == 'self', final_df.columns.get_loc('job')] = 'actor'
final_df.iloc[final_df['job'] == 'actress', final_df.columns.get_loc('job')] = 'actor'
final_df['job'].value_counts()

In [ ]:
# Convert Column Names
final_df.rename(columns = {'genres':'Genres', 
           'job': 'Staff_role', 
           'start_year': 'Released_year', 
           'title': 'Title', 
           'primary_name': 'Staff_name', 
           'production_budget': 'Investment', 
           'worldwide_gross': 'Revenue',
           'month':'Released_month',
           'profit':'Profit'}, inplace = True)
final_df

In [ ]:
final_df.info()

# Genre Analysis

In [ ]:
# Import python tools for visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Genre Frequency Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Count the frequency in each genre
data = final_df['Genres'].value_counts().head(10)
# Get x and y data
x = data.index
y = data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Top 10 Genres based on frequency')
ax.set_xlabel('Genre')
ax.set_ylabel('Frequency')
# Save figure
#plt.savefig('Top Genre Frequency.png')

In [ ]:
# Genre ROI Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Calculate the mean ROI of each genre and find top 10 genre
ROI_data = final_df.groupby(['Genres']).ROI.mean()
ROI_data = ROI_data.sort_values(ascending= False).head(10)
# Get x and y data
x = ROI_data.index
y = ROI_data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Top 10 Genres based on ROI')
ax.set_xlabel('Genre')
ax.set_ylabel('ROI')
# Save figure
#plt.savefig('Top Genre ROI.png')

In [ ]:
# Genre Profit Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Calculate the mean profit of each genre and find top 10 genre
profit_data = final_df.groupby(['Genres']).Profit.mean()
profit_data = profit_data.sort_values(ascending= False).head(10)
# Get x and y data
x = profit_data.index
y = profit_data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Top 10 Genres based on Profit')
ax.set_xlabel('Genre')
ax.set_ylabel('Worldwide Profit (M)')
# Save figure
#plt.savefig('Top Genre Profit.png')

In [ ]:
# Import Seaborn for box plot
import seaborn as sns

In [ ]:
# Create a dataframe for genre with top 3 profit
top3 = final_df.loc[(final_df['Genres'] == 'Animation')|(final_df['Genres'] == 'Sci-Fi')|(final_df['Genres'] == 'Adventure')]
top3.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.set_style('whitegrid')
ax = sns.boxplot(x='Genres', y = 'Profit', data= top3)
ax.set_title('Profit Distribution for Top 3 Genre')
#plt.savefig('Box Plot Genre Profit.png')

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.boxplot(x='Genres', y = 'ROI', data= top3)
ax.set_title('ROI Distribution for Top 3 Genre')
#plt.savefig('Box Plot Genre ROI.png')

In [ ]:
def genre_profit(genre):
    """
    Function which prints out the statistical data for profit and ROI for each genre inf final_df
    if the genre name is entered in string
    """
    sample_df = final_df.loc[final_df['Genres'] == genre]
    print(genre)
    print('<Profit in Millions>')
    print(sample_df['Profit'].describe()/10**6)
    print()
    print('<ROI>')
    print(sample_df['ROI'].describe())
    print()

In [ ]:
# Find statistical data for Adventure Movies
genre_profit('Adventure')

In [ ]:
# Find statistical data for Animation Movies
genre_profit('Animation')

In [ ]:
# Find statistical data for Sci-Fi Movies
genre_profit('Sci-Fi')

# Month to Release Film

In [ ]:
# Suggest the month to release film based on ROI average per month
# Animation Film
g_df=final_df[final_df['Genres']=='Animation']
g_df = g_df.drop_duplicates('Title')
ga_data = g_df.groupby(['Released_month']).ROI.mean()
# Genre ROI Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Get x and y data
x = ga_data.index
y = ga_data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Animation Movie ROI by Month')
ax.set_xlabel('Month')
ax.set_ylabel('ROI (%)')

# Set tick labels as name of the month corresponding the number
months =['January','February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
months_num = range(1,13)

ax.set_xticks(months_num)
ax.set_xticklabels(months)
# Save figure
#plt.savefig('Anime Monthly ROI.png')

In [ ]:
# Suggest the month to release film based on ROI average per month
# Adventure Film
g_df=final_df[final_df['Genres']=='Adventure']
g_df = g_df.drop_duplicates('Title')
ga_data = g_df.groupby(['Released_month']).ROI.mean()
# Genre ROI Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Get x and y data
x = ga_data.index
y = ga_data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Adventure Movie ROI by Month')
ax.set_xlabel('Month')
ax.set_ylabel('ROI (%)')

# Set tick labels as name of the month corresponding the number
months =['January','February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
months_num = range(1,13)

ax.set_xticks(months_num)
ax.set_xticklabels(months)
# Save figure
#plt.savefig('Adventure Monthly ROI.png')

In [ ]:
# Suggest the month to release film based on ROI average per month
# Sci-Fi Film
g_df=final_df[final_df['Genres']=='Sci-Fi']
g_df = g_df.drop_duplicates('Title')
ga_data = g_df.groupby(['Released_month']).ROI.mean()
# Genre ROI Analysis
# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))
# Get x and y data
x = ga_data.index
y = ga_data.values 
# Create bar chart
ax.bar(x, y) 
# Set title and labels
ax.set_title('Sci-Fi Movie ROI by Month')
ax.set_xlabel('Month')
ax.set_ylabel('ROI (%)')

# Set tick labels as name of the month corresponding the number
months =['January','February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
months_num = range(1,13)

ax.set_xticks(months_num)
ax.set_xticklabels(months)
# Save figure
#plt.savefig('Sci-Fi Monthly ROI.png')

In [ ]:
final_df

In [ ]:
directors = final_df[final_df.Staff_role == 'director']


In [ ]:
unique_titles_df = directors.drop_duplicates(subset= ['Title'])

In [ ]:
profit_directors = unique_titles_df.groupby('Staff_name')['Profit'].sum().sort_values(ascending=False)
top_50_directors = profit_directors.nlargest(50)
top_50_directors_profit = top_50_directors.values.sum()
total_profit = unique_titles_df['Profit'].sum()
top_50_directors_profit_perc = top_50_directors_profit/total_profit
rest_profit = 1 - top_50_directors_profit_perc 

In [ ]:
unique_titles_df

In [ ]:
import matplotlib.pyplot as plt

my_data = [rest_profit, top_50_directors_profit_perc]
my_labels = 'Directors Outside the Top 50', 'Top 50 Directors by Profit'
my_colors = ['lightblue','silver']
my_explode = (0, 0.1)
plt.pie(my_data,labels = my_labels,autopct='%1.1f%%', startangle=15, shadow = True, colors=my_colors, explode=my_explode)
plt.title('Total Profit')
plt.axis('equal')
plt.show()

In [ ]:
# 3 in 1 Bar chart
# Create datasets for top 3 genres
g1_df=final_df[final_df['Genres']=='Animation']
g1_df = g1_df.drop_duplicates('Title')
g1a_data = g1_df.groupby(['Released_month']).ROI.mean()
g2_df=final_df[final_df['Genres']=='Adventure']
g2_df = g2_df.drop_duplicates('Title')
g2a_data = g2_df.groupby(['Released_month']).ROI.mean()
g3_df=final_df[final_df['Genres']=='Sci-Fi']
g3_df = g3_df.drop_duplicates('Title')
g3a_data = g3_df.groupby(['Released_month']).ROI.mean()

# Create a figure and axis
fig, ax = plt.subplots(figsize=(15,6))

# Get x and y data
x1 = g1a_data.index
y1 = g1a_data.values
x2 = g2a_data.index
y2 = g2a_data.values
x3 = g3a_data.index
y3 = g3a_data.values

# Create Animation bar chart 0.25 left to the center and assign unique color
ax.bar(x1-0.25, y1, width= 0.25, color='navy')
# Create Adventure bar chart on the center and assign unique color
ax.bar(x2, y2, width = 0.25, color='green')
# Create Sci-Fi bar chart 0.25 right to the center and assign unique color
ax.bar(x3+0.25, y3, width = 0.25, color='darkorange')

# Set tick labels as name of the month corresponding the number
months =['January','February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
months_num = range(1,13)
ax.set_xticks(months_num)
ax.set_xticklabels(months)
# Set title, labels, and legend
ax.set_title('ROI by Month')
ax.set_xlabel('Month')
ax.set_ylabel('ROI (%)')
ax.legend(["Animation", "Adventure", "Sci-Fi"])
# Save figure
plt.savefig('Monthly ROI.png')